In [29]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

def get_info(link):
    '''
    Fonction helper pour obtenir les informations d'une offre grâce au
    code HTML de la page.
    '''
    page = requests.get(link)
    soup = BeautifulSoup(page.text, 'html.parser')
    company = soup.find('span', attrs = {'class' : "sc-bdOgaJ kaknoe wui-text"}).text

    title = soup.find('h2', attrs = {'class' : "sc-bdOgaJ kEoFYF wui-text"}).text
    lieu = soup.find('span', attrs = {'class': "sc-1eoldvz-0 bZJPQK"}).text
    
    # La structure d'une offre sur Welcome to the jungle est la même mais
    # les entreprises ont le choix des informations qu'ils veulent donner
    # Il faut donc considérer les informations manquantes
    try:
        contrat = soup.find_all("div", class_="sc-epALIP fdfdXO")[0].find('span').text
    except:
        contrat = "non spécifié"
        
    try:
        if soup.find("time").has_attr('datetime'):
            debut= soup.find("time").text
    except :
        debut = "Non trouvé"
        
    divs = soup.find_all('div', attrs = {'class': "sc-epALIP doNYBI"})
    
    
    experience = None
    education = None
    for div in divs:
        if 'Expérience' in div.text:
            experience = div.text.split("Expérience :")[1].strip()
        elif 'Éducation' in div.text:
            education =  div.text.split("Éducation :")[1].strip()
            
    col = soup.find_all("div", class_="sc-epALIP fzZZzw")

    try:
        domaine = col[0].find('span').text
    except:
        domaine = "non spécifié"
    try:
        taille = col[1].find('span').text
    except:
        taille = "non spécifié"

    try :
        profile_section = soup.find(lambda tag: tag.name == 'h4' and 'Profil recherché' in tag.text)
        profil = profile_section.find_next_sibling('div').text
    except :
        profil = "non spécifié"

    try:
        descriptif_section = soup.find(lambda tag: tag.name == 'h4' and 'Descriptif du poste' in tag.text)
        descriptif = descriptif_section.find_next_sibling('div').text
    except:
        descriptif = "non spécifié"
    
    line=[company,domaine,taille,lieu,title,debut,contrat,education,experience,descriptif,profil,link]
    return line


In [50]:
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.firefox.service import Service
from webdriver_manager.firefox import GeckoDriverManager
from bs4 import BeautifulSoup
import time

jobs_dict = {}

# On a besoin de Selenium (et pas seulement de BeautifulSoup/Requests) car
# la page de recherche de job est chargée dynamiquement avec des éléments JS

# La section suivante est adaptée pour Firefox
def crawler(role):

    opts = Options()
    opts.headless = True  # Headless mode (exécution + rapide)
    driver = webdriver.Firefox(options=opts, service=Service(GeckoDriverManager().install()))

    jobs_list = []
    
    
    n_pages = 2
    i = 1
    while i <= n_pages:
        
        role_adjusted = role
        all_jobs = f"https://www.welcometothejungle.com/fr/jobs?refinementList%5Boffices.country_code%5D%5B%5D=FR&query={role}&page="+str(i)
        driver.get(all_jobs)
        time.sleep(5) # On attend que toutes les offres soient chargées
        soup = BeautifulSoup(driver.page_source,'html.parser')
        divs = soup.find_all('div', attrs = {'class': "sc-bXCLTC bAjxyY"})
        if i == 1:
            # On obtient le nombre de pages à la première itération
            # en regardant le numéro de la dernière page en bas de l'écran
            n_pages = int(soup.find_all('a', attrs = {'class': "sc-edKZPI kCGopn"})[-1].text)
            print(n_pages)
            
        for div in divs :
            a = div.find("a")
            jobs_list.append(a["href"])
            
        i+=1


    jobs_list = list(set(jobs_list))
    jobs_dict[role] = jobs_list
    
    # Fermer le navigateur une fois terminé
    driver.quit()
    
role_list = ['deep learning','data scientist', 'data', 'ml'] # Exemple de jobs
for role in role_list:
    crawler(role)


5
12
34
6


{'deep learning': ['/fr/companies/mp-data/jobs/data-scientist-data-engineer?q=7453a57724c477d18f8a3822d4c251f5&o=c2a65ca8-cf98-40c9-879e-509e22272540',
  '/fr/companies/datascientest/jobs/data-scientist-confirme-spanish-fluent_puteaux?q=1eb2a2ac98e845892afebe1507081fb1&o=7f9fb7a3-a1ad-4e6c-b5d2-f88e80f8d26f',
  '/fr/companies/qantev/jobs/data-scientist_paris_QANTE_P4e76qx?q=7453a57724c477d18f8a3822d4c251f5&o=45736ec2-3bdc-44be-9732-14a6504ad55b',
  '/fr/companies/qantev/jobs/lead-data-scientist_paris?q=1eb2a2ac98e845892afebe1507081fb1&o=25ea0f01-813e-43f2-950c-7d3816f99b00',
  '/fr/companies/biologique-recherche-fr/jobs/chef-de-projets-r-d-machine-ingenieur_suresnes_BR_LNrpkJG?q=1eb2a2ac98e845892afebe1507081fb1&o=1ca9a4ca-a376-43c4-a9d4-994271bb391e',
  '/fr/companies/qantev/jobs/devsecops-engineer_paris_QANTE_WoQXNgr?q=1eb2a2ac98e845892afebe1507081fb1&o=5f887d27-1fcf-422c-bd6d-0f709b6687d0',
  '/fr/companies/qantev/jobs/engagement-manager-delivery-lead_paris?q=1eb2a2ac98e845892afebe15

In [60]:
base = "https://www.welcometothejungle.com"

# On crée les dataframe correspondant à chaque intitulé de poste
# Important: 'data' comprend aussi 'data scientist' et tout autre intitulé
# qui comprend le mot 'data'.

for role in jobs_dict.keys():
    df = pd.DataFrame(columns=["Company","Domaine","Taille","Lieu","Titre","Debut","Contrat","Education","Experience","Descriptif","Profil", "Link"])
    jobs_list = jobs_dict[role]
    for i,job in enumerate(jobs_list) :
        row = get_info(base+job)
        a = pd.DataFrame([row],columns=["Company","Domaine","Taille","Lieu","Titre","Debut","Contrat","Education","Experience","Descriptif","Profil","Link"])
        df = pd.concat([df,a],ignore_index=True)
    csv_file = f'welcome_to_the_jungle_data_{role}_{datetime.now()}.csv'
    df.to_csv(r'../datasets_wttj/' + csv_file) 


                                       Company  \
0                                      MP DATA   
1                                Datascientest   
2                                       Qantev   
3                                       Qantev   
4                         BIOLOGIQUE RECHERCHE   
..                                         ...   
126                                     SYSNAV   
127  RTE, Le réseau de transport d'électricité   
128                                     Qantev   
129                                     Qantev   
130                                    ARISTID   

                                               Domaine               Taille  \
0    Intelligence artificielle / Machine Learning, ...   100 collaborateurs   
1             SaaS / Cloud Services, EdTech, Formation   130 collaborateurs   
2    Intelligence artificielle / Machine Learning, ...    46 collaborateurs   
3    Intelligence artificielle / Machine Learning, ...    46 collaborateurs   
4   

                           Company  \
0                     Extracadabra   
1                  Health Data Hub   
2                      FINGREEN AI   
3                  Foxintelligence   
4                           Abbeal   
...                            ...   
1001              Société Générale   
1002                           AXA   
1003                       Natixis   
1004  Caisse d’Epargne Rhône Alpes   
1005                       Polynom   

                                                Domaine  \
0                          Application mobile, FoodTech   
1     Intelligence artificielle / Machine Learning, ...   
2     Intelligence artificielle / Machine Learning, ...   
3            Stratégie, SaaS / Cloud Services, Big Data   
4               IT / Digital, Incubateur / Accélérateur   
...                                                 ...   
1001               Banque, FinTech / InsurTech, Finance   
1002             Banque, Assurance, FinTech / InsurTech   
1003        